In [2]:
# Dowloading pyspark
#!pip install pyspark

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

PREPARE THE ENVIRONMENT, UPLOAD DATA, PREPROCESS DATA AND CREATE THE TABLES: Author, Paper, Affiliation, Book, Journal and Conference

In [4]:
# With sparkSession we create a connection to our database
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, TimestampType
from pyspark.sql.functions import count, col, xxhash64, collect_list, explode

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("Bibliography") \
      .getOrCreate()

#INPUT_FILE = "/content/drive/MyDrive/bibliography.json"
INPUT_FILE = "bibliography.json"
OPTIONS = {'multiline': 'true', 'allowNumericLeadingZero': 'true','timestampFormat': "yyyy-MM-dd'T'HH:mm:ss[.ZZZ'Z']"}

In [5]:
#AUTHOR TABLE
schemaAut = StructType(
            [StructField('authors', ArrayType(StructType([
                StructField('_id', StringType(), nullable = False),
                StructField('name', StringType(), True),
                StructField('email', StringType(), True),
                StructField('bio', StringType(), True),
                ])), True)
            ])

df_aut = spark.read.format('json').options(**OPTIONS).schema(schemaAut).json(INPUT_FILE)
df_aut = df_aut.select(explode(df_aut.authors))
df_aut = df_aut.withColumnRenamed("col", "authors")
df_aut = df_aut.filter(col("authors._id") != "null").select("authors._id","authors.name","authors.email", "authors.bio")
df_aut = df_aut.withColumnRenamed("_id", "author_id")
df_aut = df_aut.dropDuplicates(["author_id"])
df_aut.printSchema()
df_aut.show()

root
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- bio: string (nullable = true)



+--------------------+------------------+--------------------+--------------------+
|           author_id|              name|               email|                 bio|
+--------------------+------------------+--------------------+--------------------+
|53f3186fdabfae9a8...|   A. M. A. Hariri|a..m..a..hariride...|My name is A. M. ...|
|53f3186fdabfae9a8...|    Matthew Prowse|matthew.prowsefb@...|My name is Matthe...|
|53f31870dabfae9a8...|       Sui-ping Qi|sui-ping.qi19@gma...|My name is Sui-pi...|
|53f31871dabfae9a8...|     Renato Fabbri|renato.fabbrib7@g...|My name is Renato...|
|53f31873dabfae9a8...|   Joachim Schimpf|joachim.schimpf8a...|My name is Joachi...|
|53f31874dabfae9a8...|    E. Di Bernardo|e..di.bernardo10@...|My name is E. Di ...|
|53f31875dabfae9a8...|    Steven F. Roth|steven.f..roth46@...|My name is Steven...|
|53f31878dabfae9a8...|      Nima Zahadat|nima.zahadat3d@gm...|My name is Nima Z...|
|53f3187ddabfae9a8...|         Ke Fa Cen|ke.fa.cen23@gmail...|My name is Ke 

In [6]:
# PAPER TABLE WITHOUT PUBLICATION_ID
schemaPaper = StructType(
            [StructField('_id', StringType(), True),
             StructField('title', StringType(),True),
             StructField('keywords', ArrayType(StringType()), True),
             StructField('fos', ArrayType(StringType()), True),
             StructField('references', ArrayType(StringType()), True),
             StructField('page_start', IntegerType(), True),
             StructField('page_end', IntegerType(), True),
             StructField('lang', StringType(),True),
             StructField('doi', StringType(),True),
             StructField('url', ArrayType(StringType()),True),
             StructField('abstract', StringType(),True),
             StructField('publication_type', StringType(),True),
             StructField('date', TimestampType(), True)
            ])

df_paper = spark.read.format('json').options(**OPTIONS).schema(schemaPaper).json(INPUT_FILE)
df_paper = df_paper.withColumnRenamed("_id", "paper_id")
df_paper.printSchema()
df_paper.show()

root
 |-- paper_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- references: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- page_start: integer (nullable = true)
 |-- page_end: integer (nullable = true)
 |-- lang: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- abstract: string (nullable = true)
 |-- publication_type: string (nullable = true)
 |-- date: timestamp (nullable = true)



+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------+----+--------------------+--------------------+--------------------+----------------+-------------------+
|            paper_id|               title|            keywords|                 fos|          references|page_start|page_end|lang|                 doi|                 url|            abstract|publication_type|               date|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------+----+--------------------+--------------------+--------------------+----------------+-------------------+
|53e99784b7602d970...|Using XML to Inte...|[internet, hyperm...|[xml base, world ...|[53e9adbdb7602d97...|       167|     172|  en|10.1109/CMPSAC.20...|[http://dx.doi.or...|The eXtensible Ma...|            Book|1974-09-13 06:34:29|
|53e99784b7602d970...|               FCLOS|[molap, subsumpti...|[informa

In [12]:
# AFFILIATION TABLE
schemaAffiliation = StructType(
            [StructField('_id', StringType(), True),
             StructField('authors', ArrayType(StructType([
                    StructField('_id', StringType(), True),
                    StructField('org', StringType(), True)
             ])), True),
            ])

df_aff = spark.read.format('json').options(**OPTIONS).schema(schemaAffiliation).json(INPUT_FILE)
df_aff = df_aff.withColumnRenamed("_id", "paper_id")
df_aff = df_aff.select("paper_id", explode(df_aff.authors))
df_aff = df_aff.withColumnRenamed("col", "authors")
df_aff = df_aff.filter(col("authors._id") != "null").filter(col("paper_id") != "null").select("paper_id", "authors._id","authors.org")
df_aff = df_aff.withColumnRenamed("_id", "author_id")
df_aff = df_aff.dropDuplicates(["author_id", "paper_id"])
df_aff = df_aff.withColumnRenamed("org", "organization")
df_aff.printSchema()
df_aff.show()

root
 |-- paper_id: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- organization: string (nullable = true)



+--------------------+--------------------+--------------------+
|            paper_id|           author_id|        organization|
+--------------------+--------------------+--------------------+
|53e998c7b7602d970...|53f3186fdabfae9a8...|Department of Sta...|
|53e99827b7602d970...|53f3186fdabfae9a8...|Laboratory for Fo...|
|53e99924b7602d970...|53f31870dabfae9a8...|Henan Academy of ...|
|53e998dbb7602d970...|53f31871dabfae9a8...|Instituto de Físi...|
|53e998f6b7602d970...|53f31873dabfae9a8...|                null|
|53e998bfb7602d970...|53f31874dabfae9a8...|                null|
|53e9984bb7602d970...|53f31875dabfae9a8...|                null|
|53e998e8b7602d970...|53f31878dabfae9a8...|George Mason Univ...|
|53e99905b7602d970...|53f3187ddabfae9a8...|State Key Laborat...|
|53e998e9b7602d970...|53f31881dabfae9a8...|                null|
|53e9984fb7602d970...|53f31881dabfae9a8...|Tecnologico de Mo...|
|53e9980eb7602d970...|53f31883dabfae9a8...|University of Was...|
|53e997e9b7602d970...|53f

In [7]:
# JOURNAL TABLE
# Preprocessing of the journals for cleaning and merging the journals

journal_schema_preprocessing = StructType(
    [StructField('_id', StringType(), True),
     StructField('issn', StringType(), True),
     StructField('publisher', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('volume', IntegerType(), True),
     StructField('issue', IntegerType(), True),
     StructField('publication_type', StringType(),True)])

# Reading the json file
df_journals_to_filter = spark.read.format('json').options(**OPTIONS).schema(journal_schema_preprocessing).json(INPUT_FILE)

# Filtering and adjusting the dataframe
df_journals_to_filter = df_journals_to_filter.filter(col('publication_type') == 'Journal').filter(col('issn') != 'null').filter(col('venue') != 'null').filter(col('issue') >= 0).filter(col('volume') >= 0)
df_journals_to_filter = df_journals_to_filter.groupBy('venue', 'volume', 'issue', 'issn').agg(collect_list('publisher').alias('publishersArray'), collect_list('_id').alias('_id'), count(col('publisher'))) # count can be removed (I was interested in evaluating if the group by was meaningful)
df_journals_to_insert = df_journals_to_filter.withColumn('publisher', df_journals_to_filter['publishersArray'][0]).select('venue', 'volume', 'issue', 'publisher', 'issn', '_id')

# Adding the new column which contains the publication_identifier
df_journals = df_journals_to_insert.withColumn("publication_id", xxhash64('venue', 'volume', 'issue', 'issn'))

# Adding the foreign key to the papers
exploded_journals = df_journals.select(explode('_id'), 'publication_id')
#exploded_journals.show(truncate = False)

df_papers_in_journals = exploded_journals.join(df_paper, exploded_journals.col == df_paper.paper_id, "inner")
df_papers_in_journals = df_papers_in_journals.drop('col')

df_journals = df_journals.drop(df_journals._id)

# Visualizing the data
# print('Papers')
# df_papers_in_journals.show(truncate = False)
print('Schema of the journals')
df_journals.printSchema()
print('Journals')
df_journals.show(truncate=False)

Schema of the journals
root
 |-- venue: string (nullable = true)
 |-- volume: integer (nullable = true)
 |-- issue: integer (nullable = true)
 |-- publisher: string (nullable = true)
 |-- issn: string (nullable = true)
 |-- publication_id: long (nullable = false)

Journals
+-----------------------+------+-----+------------------------------------------------------------------+---------+--------------------+
|venue                  |volume|issue|publisher                                                         |issn     |publication_id      |
+-----------------------+------+-----+------------------------------------------------------------------+---------+--------------------+
|4OR                    |2     |4    |Inderscience Publishers                                           |1614-2411|-8668689828003778653|
|4OR                    |3     |1    |Accent Social and Welfare Society                                 |1614-2411|-6055461944651662439|
|4OR                    |4     |1    |Inn

In [8]:
# BOOK TABLE
# Preprocessing of the books for cleaning and merging the books
book_schema_preprocessing = StructType(
    [StructField('_id', StringType(), True),
     StructField('isbn', StringType(), True),
     StructField('publisher', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('publication_type', StringType(),True)])

# Reading the json file
df_books_to_filter = spark.read.format('json').options(**OPTIONS).schema(book_schema_preprocessing).json(INPUT_FILE)

# Filtering and adjusting the dataframe
df_books_to_filter = df_books_to_filter.filter(col('publication_type') == 'Book').filter(col('isbn') != 'null').filter(col('venue') != 'null')
df_books_to_filter = df_books_to_filter.groupBy('isbn', 'venue').agg(collect_list('publisher').alias('publishersArray'), collect_list('_id').alias('_id'), count(col('publisher'))) # count can be removed (I was interested in evaluating if the group by was meaningful)
dfbooks_to_insert = df_books_to_filter.withColumn('publisher', df_books_to_filter['publishersArray'][0]).select('venue', 'isbn', 'publisher', '_id')

# Adding the new column which is the id
df_books = dfbooks_to_insert.withColumn('publication_id', xxhash64('isbn', 'venue'))

# Adding the foreign key to the papers
exploded_books = df_books.select(explode('_id'), 'publication_id')
# exploded_books.show(truncate = False)

df_papers_in_books = exploded_books.join(df_paper, exploded_books.col == df_paper.paper_id)
df_papers_in_books = df_papers_in_books.drop('col')

df_books = df_books.drop(df_books._id)

# Visualizing the data
# print('Papers')
# df_papers_in_books.show(truncate = False)
print('Schema of the books')
df_books.printSchema()
print('Books')
df_books.show(truncate=False)

Schema of the books
root
 |-- venue: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publication_id: long (nullable = false)

Books
+------------------------------------------------------------------------------------+-------------+--------------------------------------------------+--------------------+
|venue                                                                               |isbn         |publisher                                         |publication_id      |
+------------------------------------------------------------------------------------+-------------+--------------------------------------------------+--------------------+
|ACM SIGSOFT Software Engineering Notes                                              |-159593-125-2|AGH University of Science and Technology          |-4146681027596907540|
|Theor. Comput. Sci.                                                                 |0-0304-3975  |Elsevier         

In [9]:
# CONFERENCE TABLE
# Preprocessing of the books for cleaning and merging the books

schemaConf = StructType(
    [StructField('_id', StringType(), True),
     StructField('location', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('publication_type', StringType(),True)])
# Reading the json file
df_conferences_to_filter = spark.read.format('json').options(**OPTIONS).schema(schemaConf).json(INPUT_FILE)

# Filtering and adjusting the dataframe
df_conferences_to_filter = df_conferences_to_filter.filter(col('publication_type') == 'Conference').filter(col('venue') != 'null')
df_conferences_to_filter = df_conferences_to_filter.groupBy('venue').agg(collect_list('location').alias('locations_array'), collect_list('_id').alias('_id'), count(col('location'))) # count can be removed (I was interested in evaluating if the group by was meaningful)
df_conferences_to_insert = df_conferences_to_filter.withColumn('location', df_conferences_to_filter['locations_array'][0]).select('venue', 'location', '_id')

# Adding the new column which is the id
df_conferences = df_conferences_to_insert.withColumn('publication_id', xxhash64('venue'))

# Adding the foreign key to the papers
exploded_conferences = df_conferences.select(explode('_id'), 'publication_id')
#exploded_conferences.show(truncate = False)

df_papers_in_conferences = exploded_conferences.join(df_paper, exploded_conferences.col == df_paper.paper_id)
df_papers_in_conferences = df_papers_in_conferences.drop('col')

df_conferences = df_conferences.drop(df_conferences._id)

# Visualizing the data
#print('Papers')
#df_papers_in_conferences.show(truncate = False)
print('Schema of the conferences')
df_conferences.printSchema()
print('Conferences')
df_conferences.show(truncate=False)

Schema of the conferences
root
 |-- venue: string (nullable = true)
 |-- location: string (nullable = true)
 |-- publication_id: long (nullable = false)

Conferences
+----------------------------------------------------------------------------------------+--------------------------+--------------------+
|venue                                                                                   |location                  |publication_id      |
+----------------------------------------------------------------------------------------+--------------------------+--------------------+
|"EDUCON                                                                                 |Moscow, Russia            |950373860555954453  |
|2012 50TH ANNUAL ALLERTON CONFERENCE ON COMMUNICATION, CONTROL, AND COMPUTING (ALLERTON)|Dublin, Ireland           |-4245717996156385657|
|2985415099                                                                              |Mexico City, Mexico       |-762166203857654039 |


In [10]:
# Merging the 3 dataframe which one contains the papers published in a specific media
df_papers = df_papers_in_books.union(df_papers_in_journals).union(df_papers_in_conferences)

# Visualizing the data
print('Papers schema')
df_papers.printSchema()
print('Papers data')
df_papers.show()

Papers schema
root
 |-- publication_id: long (nullable = false)
 |-- paper_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- references: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- page_start: integer (nullable = true)
 |-- page_end: integer (nullable = true)
 |-- lang: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- abstract: string (nullable = true)
 |-- publication_type: string (nullable = true)
 |-- date: timestamp (nullable = true)

Papers data


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------+----+--------------------+--------------------+--------------------+----------------+-------------------+
|      publication_id|            paper_id|               title|            keywords|                 fos|          references|page_start|page_end|lang|                 doi|                 url|            abstract|publication_type|               date|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------+----+--------------------+--------------------+--------------------+----------------+-------------------+
|-4336127124407324726|53e997d1b7602d970...|A fault diagnosis...|[stuck at defects...|[computer testing...|[53e9bca6b7602d97...|       494|     494|  en|10.1109/EDTC.1997...|[http://dx.doi.or...|In this paper we ...|            Book|1950-09-0

In [11]:
# For checking the result
print('Papers published in books')
df_papers.filter(col('publication_type') == 'Book').select('paper_id', 'title', 'publication_type', 'publication_id').show()
print('Papers published in journals')
df_papers.filter(col('publication_type') == 'Journal').select('paper_id', 'title', 'publication_type', 'publication_id').show()
print('Papers published in conferences')
df_papers.filter(col('publication_type') == 'Conference').select('paper_id', 'title', 'publication_type', 'publication_id').show()


Papers published in books


+--------------------+--------------------+----------------+--------------------+
|            paper_id|               title|publication_type|      publication_id|
+--------------------+--------------------+----------------+--------------------+
|53e997d1b7602d970...|A fault diagnosis...|            Book|-4336127124407324726|
|53e997e4b7602d970...|Problem Decomposi...|            Book| 4674035537543784623|
|53e997e8b7602d970...|X-tract: Structur...|            Book| 1144530160964015551|
|53e997e8b7602d970...|Cognitive agent p...|            Book|-6465152039831803131|
|53e997e9b7602d970...|Constraint based ...|            Book| 5112790730981969721|
|53e997ecb7602d970...|Automatic input r...|            Book| 1369111316993402587|
|53e997ecb7602d970...|Acceptability-ori...|            Book| 1430384888819899512|
|53e997ecb7602d970...|Anomalous Neighbo...|            Book|  710748220667054414|
|53e997f1b7602d970...|A Digital Watermark.|            Book| 8148044123726554362|
|53e997f4b7602d9

+--------------------+--------------------+----------------+--------------------+
|            paper_id|               title|publication_type|      publication_id|
+--------------------+--------------------+----------------+--------------------+
|53e99915b7602d970...|A note on robust ...|         Journal|-8668689828003778653|
|53e99984b7602d970...|A new approach fo...|         Journal|-6055461944651662439|
|53e998b0b7602d970...|Two-machine flow ...|         Journal| 4482587362010925183|
|53e9994cb7602d970...|Stochastic semide...|         Journal|-5369333412851545957|
|53e9990db7602d970...|Models and algori...|         Journal| 4907811008184705540|
|53e99952b7602d970...|Maximizing the mi...|         Journal|-7478992719207099405|
|53e99800b7602d970...|Integer extended ...|         Journal|-5578571051264951046|
|53e99858b7602d970...|Attraction probab...|         Journal| 3303577774132183121|
|53e9998bb7602d970...|Strategy vs risk ...|         Journal|-3839102222582550984|
|53e99915b7602d9

+--------------------+--------------------+----------------+--------------------+
|            paper_id|               title|publication_type|      publication_id|
+--------------------+--------------------+----------------+--------------------+
|53e99854b7602d970...|  The EOLES project.|      Conference|  950373860555954453|
|53e9989bb7602d970...|Life is engineeri...|      Conference|  950373860555954453|
|53e998bfb7602d970...|Gaining and maint...|      Conference|  950373860555954453|
|53e998c0b7602d970...|From manuals towa...|      Conference|  950373860555954453|
|53e998e9b7602d970...|Learning with com...|      Conference|  950373860555954453|
|53e99976b7602d970...|Cloud E-learning ...|      Conference|  950373860555954453|
|53e9997eb7602d970...|Motivating progra...|      Conference|  950373860555954453|
|53e99991b7602d970...|Monitoring studen...|      Conference|  950373860555954453|
|53e99998b7602d970...|OLAREX project: O...|      Conference|  950373860555954453|
|53e99859b7602d9

COMMANDS

In [13]:
# Command 2:

In [14]:
# Command 4: delete a group of rows

# Use the function year to extract the year from the timestamp
from pyspark.sql.functions import year

# Drop rows with conditions – where clause
# From 37626 to 37175 -> delete all the rows that represent papers published before 1950, because obsolete
df_papers = df_papers.where(year('date') > '1950')
df_papers.select('title', 'publication_type', 'date').orderBy('date').show()

+--------------------+----------------+-------------------+
|               title|publication_type|               date|
+--------------------+----------------+-------------------+
|Generalized one-u...|      Conference|1951-01-03 03:43:07|
|Predicting PDZ do...|         Journal|1951-01-03 03:43:07|
|A New EDI-based D...|         Journal|1951-01-03 03:43:07|
|Search-based Exec...|      Conference|1951-01-03 03:43:07|
|Multi-structural ...|         Journal|1951-01-03 03:43:07|
|Corpus-based ling...|            Book|1951-01-03 03:43:07|
|A comparative stu...|         Journal|1951-01-03 03:43:07|
|Fast Solution of ...|         Journal|1951-01-03 03:43:07|
|Local Hausdorff D...|         Journal|1951-01-03 03:43:07|
|Grammatical Evolu...|            Book|1951-01-03 03:43:07|
|Global optimizati...|         Journal|1951-01-03 03:43:07|
|Processing UML Mo...|            Book|1951-01-03 03:43:07|
|Balancing buffer ...|         Journal|1951-01-03 03:43:07|
|The Animation of ...|            Book|1

In [15]:
# Command 5: create a new column with the length of the paper (number of total pages)

df_papers_total_pages = df_papers \
    .filter((col('page_start') >= 0) & (col('page_end') >= 0) & (col('page_start') <= col('page_end'))) \
    .withColumn('total_pages', col('page_end') - col('page_start'))

df_papers_total_pages \
    .select(col('title'), col('page_start'), col('page_end'), col('total_pages')) \
    .show(5, truncate=False)

+-----------------------------------------------------------------+----------+--------+-----------+
|title                                                            |page_start|page_end|total_pages|
+-----------------------------------------------------------------+----------+--------+-----------+
|Problem Decomposition and the Learning of Skills                 |17        |31      |14         |
|X-tract: Structure Extraction from Botanical Textual Descriptions|2         |2       |0          |
|Cognitive agent programming                                      |1385      |1385    |0          |
|Constraint based vectorization                                   |195       |204     |9          |
|Automatic input rectification                                    |80        |90      |10         |
+-----------------------------------------------------------------+----------+--------+-----------+
only showing top 5 rows



QUERIES

In [16]:
#Query 1: WHERE, JOIN

venue, volume, issue = ('BMC Bioinformatics', '14', '1') 

df_papers_q1 = df_journals\
               .filter((col('venue') == venue) &
                       (col('volume') == volume) &
                       (col('issue') == issue))\
                .join(df_papers,
                      (df_journals['publication_id'] == df_papers['publication_id']) &
                        (df_papers['publication_type'] == 'Journal')
                     )

df_papers_q1.select(['paper_id', 'title']).show(truncate=60)

+------------------------+------------------------------------------------------------+
|                paper_id|                                                       title|
+------------------------+------------------------------------------------------------+
|53e997f4b7602d9701ff55f7|BioCause: Annotating and analysing causality in the biome...|
|53e99860b7602d970209d6b9|Predicting PDZ domain mediated protein interactions from ...|
|53e99867b7602d97020a2868|Jimena: efficient computing and system state identificati...|
|53e9989bb7602d97020d129a|Reconciliation-based detection of co-evolving gene families.|
|53e998c7b7602d97021001f0|SeqSIMLA: a sequence and phenotype simulation tool for co...|
|53e998cdb7602d97021066c0|Cheminformatic models based on machine learning for pyruv...|
|53e99938b7602d9702177171|SynTView - an interactive multi-view genome browser for n...|
|53e99953b7602d970219688f|CoMAGC: a corpus with multi-faceted annotations of gene-c...|
|53e9998bb7602d97021d0801|Large-

In [17]:
from pyspark.sql.functions import current_timestamp, unix_timestamp
# Query 2: WHERE, LIMIT, LIKE
# Find the papers written in the last twenty years in english whose keywords have the word \verb|artificial| inside the keywords. We require that these papers have the DOI set to a not null value.
# The results are ordered ascendengly by the date and only 20 elements are printed.

df = df_papers.withColumn('current time', current_timestamp())
df = df\
    .filter(
        (((unix_timestamp('current time') - unix_timestamp('date')) / 3600 / 24 / 365) > 50) & 
        (col('doi').isNotNull()))\
    .select('title', 'date', explode('keywords').alias('keyword'))\
    .filter(col('keyword').like('%artificial%'))\
    .distinct()\
    .sort(col('date').asc())\
    .limit(20)\
    .show(truncate = False)

+-----------------------------------------------------------------------------------------------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|title                                                                                          |date               |keyword                                                                                                                                        |
+-----------------------------------------------------------------------------------------------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|Evolved swarming without positioning information: an application in aerial communication relay |1951-02-02 18:36:03|artificial evolution                                                                             

In [18]:
# Query 4: GROUP BY, JOIN, AS
from pyspark.sql.functions import collect_set, size

df = df_aff\
    .join(df_papers, df_papers.paper_id == df_aff.paper_id, 'inner')\
    .drop(df_papers.paper_id).select('paper_id', 'organization', 'publication_type')\
    .filter((col('organization').isNotNull()) & (col('organization') != "") & (col('publication_type') == "Conference"))\
    .groupBy('organization')\
    .agg(collect_set('paper_id').alias('papers'))\
    .filter(size(col('papers')) > 10)\
    .show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
|                                      organization|                                            papers|
+--------------------------------------------------+--------------------------------------------------+
|        Carnegie Mellon University, Pittsburgh, PA|[53e998dbb7602d9702118f29, 53e9987db7602d97020b...|
|   Carnegie Mellon University, Pittsburgh, PA, USA|[53e998c7b7602d97020fdf46, 53e99905b7602d970214...|
| Georgia Institute of Technology, Atlanta, GA, USA|[53e99827b7602d9702048d5f, 53e998f6b7602d970213...|
|           Microsoft Research Asia, Beijing, China|[53e99976b7602d97021b91db, 53e9980eb7602d970202...|
|              Microsoft Research, Redmond, WA, USA|[53e99885b7602d97020bfe89, 53e99930b7602d970216...|
|National University of Singapore, Singapore, Si...|[53e99813b7602d970202da43, 53e99876b7602d97020b...|
|               Tsinghua University, Beijing, China|[53e998c7b76

In [19]:
# Query 5: WHERE, GROUP BY
# Retrieve some statistics about papers
from pyspark.sql.functions import sum, min, max, avg, format_number, variance

df_papers_total_pages.filter(year(col('date')) >= 2015) \
    .groupBy(year(col('date')).alias('year')) \
    .agg(count('paper_id').alias('total_papers'),
         sum('total_pages').alias('total_pages'),
         min('total_pages').alias('min_pages'),
         max('total_pages').alias('max_pages'),
         format_number(avg('total_pages'), 2).alias('avg_pages'),
         format_number(variance('total_pages'), 2).alias('var_pages')) \
    .sort(col('year').desc()) \
    .show()

+----+------------+-----------+---------+---------+---------+---------+
|year|total_papers|total_pages|min_pages|max_pages|avg_pages|var_pages|
+----+------------+-----------+---------+---------+---------+---------+
|2022|         289|       2918|        0|       41|    10.10|    47.79|
|2021|         455|       5881|        0|     1585|    12.93| 5,509.97|
|2020|         556|       5345|        0|      145|     9.61|    94.19|
|2019|         459|       4334|        0|      135|     9.44|    80.38|
|2018|         504|       4756|        0|      135|     9.44|    74.82|
|2017|         423|       3818|        0|       49|     9.03|    50.92|
|2016|         728|       6687|        0|       55|     9.19|    45.74|
|2015|         530|       5364|        0|      158|    10.12|    92.43|
+----+------------+-----------+---------+---------+---------+---------+



In [20]:
# Query 6: GROUP BY, HAVING, AS

df_papers_q6 = df_papers\
               .select('paper_id',
                       'title',
                       explode(col('references')).alias('reference'))\
               .groupBy('reference')\
               .agg(count('paper_id').alias('references_count'))\
               .filter(col('references_count') > 30)\
               .join(df_papers,
                     col('reference') == df_papers.paper_id)\
               .sort(col('references_count').desc())\

df_papers_q6.select(['title', 'references_count']).show(truncate=False)

+-----------------------------------------------------------------+----------------+
|title                                                            |references_count|
+-----------------------------------------------------------------+----------------+
|Distinctive Image Features from Scale-Invariant Keypoints        |195             |
|A simple transmit diversity technique for wireless communications|62              |
|Light field rendering                                            |56              |
|Network information flow                                         |55              |
|Mining Sequential Patterns                                       |44              |
|Symbolic Model Checking                                          |44              |
|Geodesic Active Contour.                                         |40              |
|Differential Power Analysis                                      |38              |
+----------------------------------------------------------------

In [21]:
# Query 7: WHERE, GROUP BY, HAVING, AS


In [22]:
# Query 9: WHERE, GROUP BY, HAVING, 1 JOIN

from pyspark.sql.functions import collect_set, concat, size

df = df_journals\
    .withColumnRenamed('venue', 'venueJournals')\
    .filter((col('volume')) > 10)\
    .join(df_books,df_books.publisher == df_journals.publisher,"inner")\
    .drop(df_journals.publisher)\
    .withColumnRenamed('venue', 'venueBooks')\
    .select('venueBooks', 'venueJournals', 'publisher')\
    .dropDuplicates(['venueBooks', 'venueJournals', 'publisher'])\
    .groupBy('publisher')\
    .agg(collect_set('venueBooks').alias('books'),
         collect_set('venueJournals').alias('journals'))\
    .withColumn("total_publications_per_publisher",
                concat(col("books"), col("journals")))\
    .filter(size(col("total_publications_per_publisher")) > '500')\
    .select('publisher', "total_publications_per_publisher")\
    .show(truncate = 50)

+-----------------------------------------+--------------------------------------------------+
|                                publisher|                  total_publications_per_publisher|
+-----------------------------------------+--------------------------------------------------+
|                  Taylor and Francis Ltd.|[Special Interest Group on Software Engineering...|
|                                 Elsevier|[SAS, VAST, Focus on Scientific Visualization, ...|
|Association for Computing Machinery (ACM)|[Special Interest Group on Software Engineering...|
|                          Springer Verlag|[IEEE METRICS, ISQED, Special Interest Group on...|
+-----------------------------------------+--------------------------------------------------+



In [23]:
# Query 10: WHERE, GROUP BY, HAVING, 2 JOINs
# Retrieve authors who worked for at least 3 different organizations and have published at least 3 papers with at least 5 fos and 5 references each

from pyspark.sql.functions import approx_count_distinct

df_papers \
    .filter((size(col('fos')) >= 5) & (size(col('references')) >= 5)) \
    .join(df_aff, df_papers.paper_id == df_aff.paper_id, "inner") \
    .drop(df_aff.paper_id) \
    .join(df_aut, df_aff.author_id == df_aut.author_id, "inner") \
    .drop(df_aff.author_id) \
    .groupBy("author_id") \
    .agg(count("paper_id").alias("papers_count"),
         approx_count_distinct("organization").alias("organizations_count"),
         collect_set("name").alias("name")) \
    .filter((size("name") == 1) & (col("papers_count") >= 3)  & (col("organizations_count") >= 3)) \
    .orderBy(col("papers_count").desc(), col("organizations_count").desc()) \
    .select(explode(col("name")).alias("name"), col("papers_count"), col("organizations_count")) \
    .show(5)

22/12/08 22:46:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+------------+-------------------+
|               name|papers_count|organizations_count|
+-------------------+------------+-------------------+
|   Rachid Guerraoui|          22|                 11|
|Thomas A. Henzinger|          21|                 14|
|        Dacheng Tao|          20|                 13|
|     Moshe Y. Vardi|          20|                  7|
|    Thomas S. Huang|          19|                 10|
+-------------------+------------+-------------------+
only showing top 5 rows

